In [2]:
from typing import Union

import torch
import ai_edge_torch
import torchvision
from torch.fx import GraphModule

import gorillatracker.quantization.performance_evaluation as performance_evaluation
import gorillatracker.quantization.quantization_functions as quantization_functions
from gorillatracker.datasets.cxl import CXLDataset
from gorillatracker.model import BaseModule
from gorillatracker.quantization.export_model import convert_model_to_onnx
from gorillatracker.quantization.utils import get_model_input, log_model_to_file
from gorillatracker.utils.embedding_generator import get_model_for_run_url

save_quantized_model = False
load_quantized_model = False
save_model_architecture = False
number_of_calibration_images = 1
dataset_path = "/workspaces/gorillatracker/data/splits/ground_truth-cxl-face_images-openset-reid-val-0-test-0-mintraincount-3-seed-42-train-50-val-25-test-25"
model_wandb_url = (
    "https://wandb.ai/gorillas/Embedding-EfficientNet-CXL-OpenSet/runs/famq71r6/workspace?nw=nwuserkajohpi"
)


# 1. Quantization

calibration_input_embeddings, _ = get_model_input(
    CXLDataset, dataset_path=dataset_path, partion="train", amount_of_tensors=number_of_calibration_images
)

model: BaseModule = get_model_for_run_url(model_wandb_url)

edge_model = ai_edge_torch.convert(model.eval(), (calibration_input_embeddings,))
edge_model.export("quantized_model.tflite")


ParseResult(scheme='https', netloc='wandb.ai', path='/gorillas/Embedding-EfficientNet-CXL-OpenSet/runs/famq71r6/workspace', params='', query='nw=nwuserkajohpi', fragment='') ['', 'gorillas', 'Embedding-EfficientNet-CXL-OpenSet', 'runs', 'famq71r6', 'workspace'] /gorillas/Embedding-EfficientNet-CXL-OpenSet/runs/famq71r6/workspace
Using model from run: 261-add-the-ability-to-do-quantization-using-pytorch-2-export-quantization-2024-05-21-07-13-45
Config: {'s': 64, 'seed': 42, 'beta1': 0.9, 'beta2': 0.999, 'debug': False, 'kfold': False, 'end_lr': 1e-06, 'margin': 1, 'resume': False, 'compile': False, 'delta_t': 50, 'epsilon': 1e-07, 'l2_beta': 0.01, 'offline': False, 'plugins': None, 'use_ssl': False, 'workers': 16, 'data_dir': '/workspaces/gorillatracker/data/splits/ground_truth-cxl-face_images-openset-reid-val-0-test-0-mintraincount-3-seed-42-train-50-val-25-test-25', 'l2_alpha': 0.1, 'only_val': False, 'profiler': None, 'run_name': '261-add-the-ability-to-do-quantization-using-pytorch-

wandb: Downloading large artifact model-famq71r6:v0, 1348.64MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.3


INFO:tensorflow:Assets written to: /tmp/tmprvyrjkkn/assets


INFO:tensorflow:Assets written to: /tmp/tmprvyrjkkn/assets
W0000 00:00:1716985182.107825  597074 tf_tfl_flatbuffer_helpers.cc:392] Ignored output_format.
W0000 00:00:1716985182.107847  597074 tf_tfl_flatbuffer_helpers.cc:395] Ignored drop_control_dependency.
2024-05-29 12:19:42.108050: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmprvyrjkkn
2024-05-29 12:19:42.127807: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2024-05-29 12:19:42.127824: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmprvyrjkkn
2024-05-29 12:19:42.410449: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2024-05-29 12:19:44.549150: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmprvyrjkkn
2024-05-29 12:19:44.966639: I tensorflow/cc/saved_model/loader.cc:462] SavedModel load for tags { serve }; Status: success: OK. Took 285859

In [4]:
sample_inputs = (calibration_input_embeddings,)

In [17]:

type(quantized_model.eval())
print(quantized_model.meta)

{'_user_preserved_attributes': {}, '_observed_graph_module_attrs': ObservedGraphModuleAttrs(node_name_to_qconfig={}, node_name_to_scope={'arg0_1': ('', <class 'NoneType'>), '_param_constant0': ('conv_stem', <class 'torch.nn.modules.conv.Conv2d'>), 'conv2d': ('conv_stem', <class 'torch.nn.modules.conv.Conv2d'>), 'empty': ('bn1', <class 'timm.layers.norm_act.BatchNormAct2d'>), '_param_constant1': ('bn1', <class 'timm.layers.norm_act.BatchNormAct2d'>), '_param_constant2': ('bn1', <class 'timm.layers.norm_act.BatchNormAct2d'>), '_tensor_constant0': ('bn1', <class 'timm.layers.norm_act.BatchNormAct2d'>), '_tensor_constant1': ('bn1', <class 'timm.layers.norm_act.BatchNormAct2d'>), '_native_batch_norm_legit_no_training': ('bn1', <class 'timm.layers.norm_act.BatchNormAct2d'>), 'getitem': ('bn1', <class 'timm.layers.norm_act.BatchNormAct2d'>), 'getitem_1': ('bn1', <class 'timm.layers.norm_act.BatchNormAct2d'>), 'getitem_2': ('bn1', <class 'timm.layers.norm_act.BatchNormAct2d'>), 'silu_': ('act'

In [12]:
import io
scripted = torch.jit.script(quantized_model)
b = io.BytesIO()
torch.jit.save(scripted, b)
b.seek(0)
scripted_quantized = torch.jit.load(b)

edge_model = ai_edge_torch.convert(quantized_model.eval(), sample_inputs)
edge_model.export("quantized_model.tflite")

/opt/conda/envs/research/lib/python3.10/site-packages/torch/jit/_check.py:177: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn(


OSError: Failed to get source for <function TreeSpec.__init__ at 0x7fe2478191b0> using inspect.getsource